In [582]:
import pandas as pd
# from fbprophet import Prophet
import pandas
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import numpy as np
from praveganb.pravega_stream import UnindexedStream
import seaborn as sns
import pytz
import grpc
import imp
import pravega.grpc_gateway as pravega
from datetime import datetime, timedelta
import json
%matplotlib inline

# Setup the Connection with pravega and Read System Usage

In [583]:
gateway = '10.243.54.104:80'
scope = 'alerts'
stream = 'snmp-enriched'

In [584]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
snmpdata = UnindexedStream(scope='alerts', stream='snmp-enriched', pravega_client=pravega_client)
events = snmpdata.read_events_from_stream(stop_at_tail=True)

In [ ]:
df = pd.DataFrame(list(events))

In [ ]:
df['timeStrStamp'] = pandas.to_datetime(df['receivedTimestamp'],unit="ms",utc=True)

In [ ]:
minTime = (datetime.now() - timedelta(hours = 3)).timestamp() * 1000

In [ ]:
df = df[(df.receivedTimestamp > minTime )]

# Notify Level for Past Two Hours

In [ ]:
df.notifyLevel.value_counts()

# Notify Type and Message Breakdown By Device for Past Two Hours

In [ ]:
df[['deviceMibModule','notifyLevel', 'notifyMessage']].value_counts()

# Notify Message for Past Two Hours

In [ ]:
mess = df.notifyMessage.value_counts()

In [ ]:
mess.plot(kind='bar', figsize=(20,10))

# Device Type for Past Two Hours

In [ ]:
dev = df.deviceMibModule.value_counts()

In [ ]:
dev.plot(kind='bar', figsize=(20,10))

# SNMP Trap Type  Breakdown by Device for Past Two Hours

In [ ]:
expand_df = pd.json_normalize(df['trap'], meta=['receivedTimestamp'])

In [ ]:
df = df.join(expand_df).drop('trap', axis=1)

In [ ]:
df[['type', 'deviceMibModule']].value_counts()

In [ ]:
df[['type', 'deviceMibModule']].value_counts()